In [1]:
import multicelltypist
from datetime import date
import hisepy
import numpy as np
import os
import pandas as pd
import scanpy as sc

In [2]:
def read_adata_uuid(h5ad_uuid):
    h5ad_path = '/home/jupyter/cache/{u}'.format(u = h5ad_uuid)
    if not os.path.isdir(h5ad_path):
        hise_res = hisepy.reader.cache_files([h5ad_uuid])
    h5ad_filename = os.listdir(h5ad_path)[0]
    h5ad_file = '{p}/{f}'.format(p = h5ad_path, f = h5ad_filename)
    adata = sc.read_h5ad(h5ad_file)
    return adata

In [3]:
def resample_anndata_min_max(adata, label_column, max_cells=None, min_cells=None, random_state = 3030):
    """
    Resamples an AnnData object based on the cell labels, with the option to resample with 
    replacement for classes below a specified threshold.

    Parameters:
    ad (AnnData): The AnnData object to be resampled.
    label_column (str): The column in ad.obs where the labels are stored.
    max_cells (int, optional): The maximum number of cells to keep per label. If None, no upper limit is applied.
    min_cells (int, optional): The minimum number of cells below which resampling with replacement occurs. If None, no lower limit is applied.
    random_state (int, default = 3030): An integer used to set the state of the numpy.random.Generator
    
    Returns:
    AnnData: The resampled AnnData object.
    """
    
    labels = adata.obs[label_column].unique()

    subsets = []

    rng = np.random.default_rng(random_state)
        
    for label in labels:
        # Subset AnnData object for the current label
        subset = adata.obs[adata.obs[label_column] == label]
    
        # Resample with replacement if the number of cells is below min_cells and min_cells is defined
        if min_cells is not None and subset.shape[0] < min_cells:
            subset = subset.sample(min_cells, replace = True, random_state = rng)
        # Resample without replacement if the number of cells is greater than max_cells and max_cells is defined
        elif max_cells is not None and subset.shape[0] > max_cells:
            subset = subset.sample(max_cells, replace = False, random_state = rng)
    
        subsets.append(subset)

    # Concatenate all subsets
    resampled_obs = pd.concat(subsets)
    
    resampled_adata = adata[resampled_obs.index]
    resampled_adata.obs_names_make_unique()

    return resampled_adata

In [4]:
label_column = 'AIFI_L1'
max_cell_number = 100000

## Read clean, annotated dataset

In [5]:
h5ad_uuid = '6e8972a5-9463-4230-84b4-a20de055b9c3'

In [6]:
adata = read_adata_uuid(h5ad_uuid)

In [7]:
adata.shape

(1823666, 1261)

In [8]:
adata.obs[label_column].value_counts()

AIFI_L1
T cell             1152286
Monocyte            327919
B cell              160632
NK cell             147761
DC                   23287
Platelet              7903
Progenitor cell       1526
Erythrocyte           1508
ILC                    844
Name: count, dtype: int64

## Sample and prepare reference data

In [9]:
adata_subset = resample_anndata_min_max(
    adata, 
    label_column, 
    max_cells = max_cell_number,
    random_state = 3030
)

In [10]:
adata_subset.shape

(435068, 1261)

In [11]:
adata_subset.obs[label_column].value_counts()

AIFI_L1
B cell             100000
Monocyte           100000
NK cell            100000
T cell             100000
DC                  23287
Platelet             7903
Progenitor cell      1526
Erythrocyte          1508
ILC                   844
Name: count, dtype: int64

In [12]:
adata_subset = adata_subset.raw.to_adata()
adata_subset.shape

(435068, 33538)

In [13]:
sc.pp.normalize_total(adata_subset, target_sum=1e4)
sc.pp.log1p(adata_subset)

## Generate Initial model

In [14]:
model_fs = multicelltypist.train(
    adata_subset, 
    label_column, 
    n_jobs = 60, 
    max_iter = 10, 
    multi_class = 'ovr', 
    use_SGD = True
)

🍳 Preparing data before training
✂️ 5204 non-expressed genes are filtered out
🔬 Input data has 435068 cells and 28334 genes
⚖️ Scaling input data
🏋️ Training data using SGD logistic regression
⚠️ Warning: it may take a long time to train this dataset with 435068 cells and 28334 genes, try to downsample cells and/or restrict genes to a subset (e.g., hvgs)
✅ Model training done!


## Identify top features used for the model

Detected genes:

In [15]:
df = adata_subset.X.toarray()
flag = df.sum(axis = 0) == 0
gene = adata_subset.var_names[ ~flag]

Features with high absolute classifier coefficients for each cell type class

`np.argpartition` will take the coefficient scores for each class, and retrieve the positions of the highest absolute coefficient scores to the end of an array of positions. We then select the `top_n` positions from the end of our array of positions, which allow us to retrieve genes with the highest absolute coefficients for each class.

We can then combine these to get a unique list of genes that are important for our model.

In [16]:
top_n = 200

gene_index = np.argpartition(
    np.abs(model_fs.classifier.coef_),
    -top_n,
    axis = 1
)
gene_index = gene_index[:, -top_n:]
gene_index = np.unique(gene_index)

print('Number of genes selected: {n}'.format(n = len(gene_index)))

Number of genes selected: 1102


In [17]:
selected_genes = gene[gene_index.tolist()]
selected_df = pd.DataFrame({'gene': selected_genes})

In [18]:
selected_df.head()

,gene
0,HES4
1,TTLL10
2,TNFRSF18
3,TNFRSF4
4,RPL22


## Generate full model using selected features

In [19]:
adata = adata.raw.to_adata()

In [20]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [21]:
adata = adata[:, adata.var_names.isin(selected_genes)]

In [22]:
adata.shape

(1823666, 1102)

In [23]:
model_fs = multicelltypist.train(
    adata, 
    label_column, 
    n_jobs = 60,
    max_iter = 100,
    multi_class = 'ovr',
    check_expression = False
)

🍳 Preparing data before training
🔬 Input data has 1823666 cells and 1102 genes
⚖️ Scaling input data
🏋️ Training data using logistic regression
✅ Model training done!


## Write outputs for storage

In [24]:
out_dir = 'output'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [25]:
out_genes = 'output/ref_pbmc_clean_celltypist_top{n}_features_AIFI_L1_{d}.csv'.format(
    n = top_n,
    d = date.today()
)

selected_df.to_csv(out_genes)

In [26]:
out_model = 'output/ref_pbmc_clean_celltypist_model_AIFI_L1_{d}.pkl'.format(
    d = date.today()
)

model_fs.write(out_model)

## Upload model to HISE

Finally, we'll use `hisepy.upload.upload_files()` to send a copy of our output to HISE to use for downstream analysis steps.

In [27]:
study_space_uuid = '64097865-486d-43b3-8f94-74994e0a72e0'
title = 'PBMC Reference AIFI_L1 CellTypist Model {d}'.format(d = date.today())

In [28]:
in_files = [h5ad_uuid]

In [29]:
in_files

['6e8972a5-9463-4230-84b4-a20de055b9c3']

In [30]:
out_files = [out_genes, out_model]

In [31]:
out_files

['output/ref_pbmc_clean_celltypist_top200_features_AIFI_L1_2024-03-09.csv',
 'output/ref_pbmc_clean_celltypist_model_AIFI_L1_2024-03-09.pkl']

In [32]:
hisepy.upload.upload_files(
    files = out_files,
    study_space_id = study_space_uuid,
    title = title,
    input_file_ids = in_files
)

output/ref_pbmc_clean_celltypist_top200_features_AIFI_L1_2024-03-09.csv
output/ref_pbmc_clean_celltypist_model_AIFI_L1_2024-03-09.pkl
you are trying to upload file_ids... ['output/ref_pbmc_clean_celltypist_top200_features_AIFI_L1_2024-03-09.csv', 'output/ref_pbmc_clean_celltypist_model_AIFI_L1_2024-03-09.pkl']. Do you truly want to proceed?


(y/n) y


{'trace_id': '404e81d5-4afd-49c5-a973-0dfedff8dd73',
 'files': ['output/ref_pbmc_clean_celltypist_top200_features_AIFI_L1_2024-03-09.csv',
  'output/ref_pbmc_clean_celltypist_model_AIFI_L1_2024-03-09.pkl']}

In [33]:
import session_info
session_info.show()